# Implementing an Physics-informed neural network for the 1D Schrodinger equation using the PINN framework

### Including necessary libaries 

In [1]:
from PINNFramework.PINN import Interface
from torch.autograd import grad
import torch
import numpy as np
import torch.nn as nn
import scipy.io
from pyDOE import lhs
import torch.optim as optim

### Underlying PDE
$f:=i h_{t}+0.5 h_{x x}+|h|^{2} h$

### Implementing needed functions

In [2]:
class SchrodingerPINN(Interface):
    def __init__(self, model, input_d, output_d, lb, ub):
        super().__init__(model,input_d,output_d)
        self.lb = lb
        self.ub = ub
        
    def pde(self, x, u, derivatives):
        u_xx = derivatives[:,0]
        v_xx = derivatives[:,1]
        _u = u[:,0]
        _v = u[:,1]
        real_part = - 0.5 * v_xx - (_u**2 - _v**2)*_v
        imaginary_part= 0.5 * u_xx + (_u**2 + _v**2)*_u 
        result = torch.stack([real_part,imaginary_part],1)
        return result
        
    def derivatives(self, u, x):
        grads= torch.ones(x.shape[0])
        pred_u = u[:,0]
        pred_v = u[:,1]
        J_u = grad(pred_u, x, create_graph=True, grad_outputs=grads)[0]
        J_v = grad(pred_v, x, create_graph=True, grad_outputs=grads)[0]
        
        #calculate first order derivatives
        u_x = J_u[:,0]
        u_t = J_u[:,1]

        v_x = J_v[:,0]
        v_t = J_v[:,1]
        
        # calculate second order derivatives
        J_u_x = grad(u_x, x, create_graph=True, grad_outputs=grads)[0]
        J_v_x = grad(v_x, x, create_graph=True, grad_outputs=grads)[0]

        u_xx = J_u_x[:,0]
        v_xx = J_v_x[:,0]
        pred_derivatives = torch.stack([u_xx,v_xx,u_t,v_t],1)
        return pred_derivatives
    
        
    def input_normalization(self,x):
        """
        Implementation of min-max scaling in range of [-1,1]
        """
        return 2.0 * (x - self.lb) / (self.ub - self.lb) - 1.0

###  Creating a model with the sequential API from torch

In [3]:
pinn_model = nn.Sequential(
          nn.Linear(2,100),
          nn.Tanh(),
          nn.Linear(100,100),
          nn.Tanh(),
          nn.Linear(100,2)
        )

In [4]:
lb = torch.tensor([-5.0, 0.0])
ub = torch.tensor([[5.0, np.pi / 2]])

In [5]:
model = SchrodingerPINN(model = pinn_model, input_d = 2, output_d = 2, lb = lb, ub= ub)

### Testing forward function of the model (testing normalization)

In [6]:
sample_x = torch.randn(100,2)
sample_pred = model(sample_x)
sample_pred.shape

torch.Size([100, 2])

### Preparing data (implement your dataset here)

In [7]:
noise = 0.0
np.random.seed(1234)

# Doman bounds
lb = np.array([-5.0, 0.0])  # lower bound consists of [lower bound of x, lower bound of t]
ub = np.array([5.0, np.pi / 2])  # upper bound follows from lower bound

# defines the sizes of the neural network
N0 = 50
N_b = 50
N_f = 20000

data = scipy.io.loadmat('/home/stille15/AIPP/Data/NLS.mat')


t = data['tt'].flatten()[:, None]  # get timestamps
x = data['x'].flatten()[:, None]  # get x positions
Exact = data['uu']
# definie labels
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)
Exact_h = np.sqrt(Exact_u ** 2 + Exact_v ** 2)

X, T = np.meshgrid(x, t)

X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))  # concats the arrays
u_star = Exact_u.T.flatten()[:, None]  #
v_star = Exact_v.T.flatten()[:, None]
h_star = Exact_h.T.flatten()[:, None]

###########################

idx_x = np.random.choice(x.shape[0], N0, replace=False)
idx_x = np.sort(idx_x)

x0 = x[idx_x, :]
u0 = Exact_u[idx_x, 0:1]
v0 = Exact_v[idx_x, 0:1]

idx_t = np.random.choice(t.shape[0], N_b, replace=False)
idx_t = np.sort(idx_t)
tb = t[idx_t, :]

x_f = lb + (ub - lb) * lhs(2, N_f) # determine sampling points 
t0 = torch.zeros([x0.shape[0],1])

X_lb = np.concatenate((0 * tb + lb[0], tb), 1)  # (lb[0], tb)
X_ub = np.concatenate((0 * tb + ub[0], tb), 1)  # (ub[0], tb)

x_b = np.vstack((X_lb,X_ub)) # [x,t]
x_0 = np.concatenate([x0,t0],1)
u_b = np.zeros(x_b.shape)
u_0 = np.concatenate([u0,v0],1)




### Create input_data dictionary and transfer data to torch

In [8]:
x = {"x_0": torch.tensor(x_0).float(), "x_b": torch.tensor(x_b).float(), "x_f":torch.tensor(x_f).float()}
u_0 = torch.tensor(u_0).float()
u_b = torch.tensor(u_b).float()

### Preparing Optimizer for training


In [9]:
optimizer = optim.Adam(model.parameters(),lr=1e-3)

### Training_loop 

In [10]:
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    loss = model.pinn_loss(x, u_0, u_b,interpolation_criterion=nn.MSELoss(), boundary_criterion=nn.MSELoss(), pde_norm=nn.MSELoss())
    loss.backward()
    print("Epoch %d Loss %.10f:"%(epoch + 1, loss.item()))
    optimizer.step()

Epoch 1 Loss 17.6609230042:
Epoch 2 Loss 16.6173038483:
Epoch 3 Loss 15.6132755280:
Epoch 4 Loss 14.6480093002:
Epoch 5 Loss 13.7200527191:
Epoch 6 Loss 12.8274488449:
Epoch 7 Loss 11.9682512283:
Epoch 8 Loss 11.1406841278:
Epoch 9 Loss 10.3431282043:
Epoch 10 Loss 9.5742053986:
Epoch 11 Loss 8.8328800201:
Epoch 12 Loss 8.1185379028:
Epoch 13 Loss 7.4310216904:
Epoch 14 Loss 6.7706074715:
Epoch 15 Loss 6.1379303932:
Epoch 16 Loss 5.5339021683:
Epoch 17 Loss 4.9596123695:
Epoch 18 Loss 4.4162387848:
Epoch 19 Loss 3.9049463272:
Epoch 20 Loss 3.4268074036:
Epoch 21 Loss 2.9827165604:
Epoch 22 Loss 2.5733277798:
Epoch 23 Loss 2.1990065575:
Epoch 24 Loss 1.8598101139:
Epoch 25 Loss 1.5554894209:
Epoch 26 Loss 1.2855019569:
Epoch 27 Loss 1.0490258932:
Epoch 28 Loss 0.8449721336:
Epoch 29 Loss 0.6719809771:
Epoch 30 Loss 0.5284147263:
Epoch 31 Loss 0.4123407602:
Epoch 32 Loss 0.3215260506:
Epoch 33 Loss 0.2534451187:
Epoch 34 Loss 0.2053197175:
Epoch 35 Loss 0.1741889119:
Epoch 36 Loss 0.1570